In [1]:
import pandas as pd

Считывание данных

In [2]:
df_departments = pd.read_csv('data/задача 3/departments.csv')
df_income = pd.read_csv('data/задача 3/income_out.csv')
df_operations = pd.read_csv('data/задача 3/operations.csv')

Объединение в таблицу

In [3]:
df = pd.merge(df_operations, df_departments,how ='left',left_on='department_id', right_on='id')
df = df[(df['month'] <= 12) & (df['day'] <= 31)]
select_columns = ['year','month','day','name','income',]
df = df[select_columns]
df.head(3)

,year,month,day,name,income
0,2021,1,4,Отдел безопасности,669306
1,2021,1,6,Отдел финансового мониторинга,806090
2,2021,1,8,Отдел безопасности,364802


Объединение в сводную

In [4]:
pivot_df = df.pivot_table(values='income', index=['year', 'month'],columns ='name', aggfunc='sum', fill_value=0)
pivot_df = pivot_df.reset_index().rename_axis(None, axis=1)
pivot_df.head(3)

,year,month,Отдел безопасности,Отдел продаж,Отдел финансового мониторинга
0,2021,1,4074097,0,3937796
1,2021,2,3701828,0,4030412
2,2021,3,6016315,0,2682574


Подготовка формата и сохранение

In [5]:
columns = ['year','month','department','income']
result_df = pd.DataFrame(columns=columns)
len_pivot = len(pivot_df)
for name in pivot_df.columns[2:]:
    year = pivot_df.year.values
    month = pivot_df.month.values
    department_name = len_pivot*[name]
    income = pivot_df[name]
    
    add_df = pd.DataFrame(dict(zip(columns,[year,month,department_name,income])))
    add_df = add_df.groupby('year').filter(lambda x: (x['income'] > 0).all()) # убираем значения простая отдела в течение года
    result_df = pd.concat([result_df,add_df])
result_df = result_df.sort_values(['year','month','department'])


In [6]:
result_df.head()

,year,month,department,income
0,2021,1,Отдел безопасности,4074097
0,2021,1,Отдел финансового мониторинга,3937796
1,2021,2,Отдел безопасности,3701828
1,2021,2,Отдел финансового мониторинга,4030412
2,2021,3,Отдел безопасности,6016315


In [7]:
result_df.to_csv('3_ex__result.csv',index=False)